note : we need all the genes to do the DE analysis yaaay 
but, i found a way to load the whole file (it was a compressing issue when creating the file after reducing the dimensions)

set the random seed

In [1]:
the.seed <-42
set.seed(the.seed)

packages installation

In [ ]:
install.packages("Seurat")
install.packages("remotes")
remotes::install_github("mojaveazure/seurat-disk")
install.packages("hdf5r")
install.packages("reticulate")
install.packages("BiocManager")
BiocManager::install(c("zellkonverter", "HDF5Array"))
BiocManager::install("muscat")

Installing package into 'C:/Users/yarad/AppData/Local/R/win-library/4.5'
(as 'lib' is unspecified)



library loading

In [1]:

library(zellkonverter)
library(Seurat)
library(SeuratDisk)
library(zellkonverter)
library(reticulate)
library(SingleCellExperiment)
library(dplyr)
library(muscat)


ERROR: Error in library(zellkonverter): there is no package called 'zellkonverter'


choose path for data files

In [ ]:
# elodie's path : 
path = "/Users/elodiehusson/Desktop/AD & PD/"

#Yara's path :
# path = "C:/Users/yarad/Desktop/x/Masters/Master BMC - Sorbonne/M2/Single Cell/Project/Coding Project/"

i use a single cell experiment object. It's not a Seurat object (too complicated !!) - maybe we can use a seurat object but it seems like it's complicated. 

In [ ]:
sce <- readH5AD(
    file.path(path, "77k_AD_PD_CTRL.h5ad"), #loading the file with our paths
    use_hdf5 = TRUE, # reduces part of what is loaded in memory (to reduce RAM usage, and time)
)

sce

anndata._core.sparse_dataset._CSRDatasetanndata._core.sparse_dataset.BaseCompressedSparseDatasetanndata.abc.CSRDatasetanndata.abc._AbstractCSDatasetabc.ABCpython.builtin.object



class: SingleCellExperiment 
dim: 34176 77052 
metadata(12): batch_condition citation ... title uid
assays(1): X
rownames(34176): ENSG00000000003 ENSG00000000005 ... ENSG00000288642
  ENSG00000288643
rowData names(8): gene_name n_cells ... feature_length feature_type
colnames(77052): Donor_333-1-GTCTGTCGTTCTCCTG-1
  Donor_1001-1-ACTTTGTGTTACCGTA-1 ... Donor_634-2-ACGATCATCATGAAAG-0
  Donor_721-1-AAGCGAGCAGCGTACC-0
colData names(36): Source n_genes ... development_stage
  observation_joinid
reducedDimNames(1): X_umap
mainExpName: NULL
altExpNames(0):

In [ ]:
# gives the normalized matrix of counts
assays(sce) 

List of length 1
names(1): X

In [13]:
logcounts(sce) <- assay(sce, "X")

In [18]:
assayNames(sce)

[1] "X"         "logcounts"

CODE CHATGPT POUR EXPRESSION DIFFERENTIELLE : D'ABORD, ON S'OCCUPE D'ORGANISER LES DONNEES DANS LE SCE
mais du coup chatgpt m'a dit que on pouvait pas faire de DESeq2, ou edgeR pq les données sont normalisées, jsp si c'est vrai. 
aussi, une autre option est de transformer le SCE en un objet Seurat, et puis continuer la DE avec. Mais là jsplus quoi faire. 
je l'ai run, et c'est très long (j'ai arreté, pq ca voulait pas s'arreter de tourner), donc jsp si c'est normal ou quoi, mais c'est le plus loin que j'ai pu aller.

In [21]:
### 1) Harmoniser les colonnes du SCE 

# group_id = la condition d'intérêt
sce$group_id <- case_when(
    sce$disease == "dementia || Alzheimer disease" ~ "AD",
    sce$disease == "dementia || Parkinson disease" ~ "PD",
    sce$disease == "normal" ~ "CTRL",
    TRUE ~ NA_character_
)

# sample_id = donneur
sce$sample_id <- as.character(sce$donor_id)

# cluster_id = type cellulaire
# (tu peux aussi choisir sce$subclass ou sce$subtype selon ta résolution)
sce$cluster_id <- as.character(sce$cell_type)


In [22]:
### 2) On ne garde que AD / PD / CTRL 
sce <- sce[, !is.na(sce$group_id)]


### 3) Vérifier que les colonnes nécessaires existent bien 
stopifnot(
  "sample_id"  %in% colnames(colData(sce)),
  "group_id"   %in% colnames(colData(sce)),
  "cluster_id" %in% colnames(colData(sce))
)


### 4) S'assurer que tout est en character 
sce$sample_id  <- as.character(sce$sample_id)
sce$group_id   <- as.character(sce$group_id)
sce$cluster_id <- as.character(sce$cluster_id)


### 5) Vérif simple
table(sce$group_id)
table(sce$cluster_id)[1:10]  # affichage partiel


### 6) (OPTIONNEL) créer un identifiant sample-group 
sce$sample_group <- paste0(sce$sample_id, "_", sce$group_id)


### 7) Mask logique (rarement utile mais ok)
mask <- !is.na(sce$sample_id) &
        !is.na(sce$group_id) &
        !is.na(sce$cluster_id)

sce <- sce[, mask]




   AD  CTRL    PD 
31949 37331  7772 


                                                     astrocyte 
                                                          9235 
                                                        B cell 
                                                             3 
                                              endothelial cell 
                                                           740 
                                              GABAergic neuron 
                                                          1183 
       L2/3 intratelencephalic projecting glutamatergic neuron 
                                                          9897 
     L2/3-6 intratelencephalic projecting glutamatergic neuron 
                                                          5546 
L5 extratelencephalic projecting glutamatergic cortical neuron 
                                                            12 
                     L5/6 near-projecting glutamatergic neuron 
                                       

In [ ]:

### 8) Pseudobulk : utiliser ASSAY = "X" ----
# car ton fichier n'a PAS de "logcounts", seulement "X"

pb <- aggregateData(
  sce,
  assay = "X",       # <- LE BON CHOIX
  fun   = "mean",    # pour log-expressions
  by    = c("cluster_id", "sample_id")
)

pb